In [ ]:
import torch
import torchtext
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.nn.functional import one_hot
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

import os
import re
import unicodedata
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%%capture
!pip install indic-nlp-library
from indicnlp.tokenize import indic_tokenize

In [ ]:
language1_filepath = ""
language2_filepath = ""
lang1 = "english"
lang2 = "hindi"

with open(language1_filepath, "r", encoding='utf-8') as f:
  language1_data = f.readlines()

with open(language2_filepath, "r", encoding='utf-8') as f:
  language2_data = f.readlines()

In [ ]:
req_language1 = language1_data[242929:267939]
req_language2 = language2_data[242929:267939]
data = {"language1_txt":req_language1,"language2_txt":req_language2}

df = pd.DataFrame(data)

In [ ]:
def is_language_corrupted(text):
    if re.search(r'[^\u0900-\u097F\s,.?!-]', text):
        return True
    return False

In [ ]:
# Cleaning up text with is_language_corrupted(text)
# Un-comment below if using Hindi or code changed for the language you are using

# df['Corrupted'] = df['language2_txt'].apply(is_language_corrupted)
# df_clean = df[~df['Corrupted']]
# df_clean = df_clean.drop(columns=['Corrupted'])

In [ ]:
def clean_text(text, language="english"):
    text = unicodedata.normalize('NFKC', text)
    if language == "english":
        text = text.lower()
    if language == "hindi":
        text = re.sub('[a-zA-Z]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
df_clean["language1_txt"] = df_clean["language1_txt"].apply(clean_text, args=(lang1,))
df_clean["language2_txt"] = df_clean['language2_txt'].apply(clean_text, args=(lang2,))

In [ ]:
df_clean['language1_Words'] = df_clean['language1_txt'].apply(lambda x: len(x.split()))
df_clean['language2_Words'] = df_clean['language2_txt'].apply(lambda x: len(x.split()))

In [ ]:
average_language1_words = df_clean['language1_Words'].quantile(.99)
average_language2_words = df_clean['language2_Words'].quantile(.99)
averages = [average_language1_words, average_language2_words]
languages = ['language1', 'language2']

plt.figure(figsize=(8, 6))
plt.bar(languages, averages, color=['blue', 'red'])
plt.xlabel('Language')
plt.ylabel('Number of Words per Sentence')
plt.title('Comparison of Number of Words Counts in language1 and language2')
plt.ylim(0, max(averages) + 1)
plt.show()

In [ ]:
tokenizer_language1 = get_tokenizer('basic_english')
tokenizer_language2 = indic_tokenize.trivial_tokenize

tokenized_language1_txt = [tokenizer_language1(language1_sen) for language1_sen in df_clean['language1_txt'] ]
tokenized_language2_txt = [tokenizer_language2(language2_sen) for language2_sen in df_clean['language2_txt'] ]

In [ ]:
features_vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_language1_txt, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
target_vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_language2_txt, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

features_vocab.set_default_index(features_vocab['<unk>'])
target_vocab.set_default_index(target_vocab['<unk>'])

In [ ]:
features_vocab_total_words = len(features_vocab)
target_vocab_total_words = len(target_vocab)

In [ ]:
def tokens_to_indices(tokenized_texts, vocab):
    indices_texts = []
    for sentence in tokenized_texts:
        indices_texts.append([vocab[token] for token in sentence if token in vocab])
    return indices_texts

In [ ]:
language1_indices = tokens_to_indices(tokenized_language1_txt, features_vocab)
language2_indices = tokens_to_indices(tokenized_language2_txt, target_vocab)

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, language1_data, language2_data):
        self.language1_data = language1_data
        self.language2_data = language2_data

    def __len__(self):
        return len(self.language1_data)

    def __getitem__(self, idx):
        language1 = torch.tensor(self.language1_data[idx], dtype=torch.long)
        language2 = torch.tensor(self.language2_data[idx], dtype=torch.long)
        return language1, language2

dataset = TranslationDataset(language1_indices, language2_indices)
FIXED_LENGTH = 60


In [ ]:
def collate_batch(batch):
    language1_batch, language2_batch = zip(*batch)

    language1_batch = [torch.tensor(seq[:FIXED_LENGTH], dtype=torch.long) if len(seq) > FIXED_LENGTH else torch.cat([torch.tensor(seq, dtype=torch.long), torch.full((FIXED_LENGTH - len(seq),), features_vocab['<pad>'], dtype=torch.long)]) for seq in language1_batch]
    language2_batch = [torch.tensor(seq[:FIXED_LENGTH], dtype=torch.long) if len(seq) > FIXED_LENGTH else torch.cat([torch.tensor(seq, dtype=torch.long), torch.full((FIXED_LENGTH - len(seq),), target_vocab['<pad>'], dtype=torch.long)]) for seq in language2_batch]

    language1_batch = torch.stack(language1_batch)
    language2_batch = torch.stack(language2_batch)
    return language1_batch, language2_batch

In [ ]:
batch_size = 32
train_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_batch, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()

        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        key_shape = keys.size()

        query = query.repeat(1, key_shape[1], 1)
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(-1)

        weights = torch.softmax(scores, dim=1)
        weights = weights.unsqueeze(1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)

        bos_token_index = features_vocab['<bos>']
        decoder_input = torch.full((batch_size, 1), bos_token_index, dtype=torch.long, device=device)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(60):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=5, plot_every=5, save_every=5, save_path=''):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

        if epoch % save_every == 0:
            encoder_save_path = os.path.join(save_path, '')
            decoder_save_path = os.path.join(save_path, '')
            torch.save(encoder.state_dict(), encoder_save_path)
            torch.save(decoder.state_dict(), decoder_save_path)
    showPlot(plot_losses)

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data
        input_tensor, target_tensor = input_tensor.to(device), target_tensor.to(device)
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
hidden_size = 128
batch_size = 32
features_vocab_total_words = len(features_vocab)
target_vocab_total_words = len(target_vocab)

encoder = EncoderRNN(features_vocab_total_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, target_vocab_total_words).to(device)

In [ ]:
train(train_dataloader, encoder, decoder, n_epochs = 50, print_every=5, plot_every=5)

In [ ]:
encoder.load_state_dict(torch.load(""))
decoder.load_state_dict(torch.load(""))

In [ ]:
def evaluate(encoder, decoder, sentence, feature_vocab, target_vocab):
    with torch.no_grad():

        tokenized_language1_txt_test = tokenizer_language1(sentence)
        language1_indices_test = tokens_to_indices(tokenized_language1_txt_test, features_vocab)
        input_tensor = torch.LongTensor(language1_indices_test).to(device).reshape(1,4)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        EOS_token = feature_vocab['<eos>']
        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<eos>')
                break
            decoded_words.append(target_vocab.get_itos()[idx] if idx < len(target_vocab) else '<unk>')
    return decoded_words, decoder_attn

In [ ]:
sentence = ""
decoder_output, attn_weights = evaluate(encoder, decoder, sentence, features_vocab, target_vocab)